In [1]:
ablation = 'ALKERec'#['ALKERec','RAD-BC','random','easiest','hardest','TR']
name = 'Games' #['Games','ML']

In [2]:

import argparse
import numpy as np
from utils.dataset import Dataset,DataIterator,get_DataLoader
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

def parse_args(name):   
    parser = argparse.ArgumentParser(description="Run .")  
    parser.add_argument('--model', nargs='?', default='STAMP_S_distill')
    parser.add_argument('--dataset', nargs='?', default=name,
                        help='Choose a dataset.')
    parser.add_argument('--batch_size', type=int, default=1024,
                        help='Batch size.')
    parser.add_argument('--hidden_factor', type=int, default=10,
                        help='Number of hidden factors.')
    parser.add_argument('--lamda', type=float, default = 10e-5,
                        help='Regularizer for bilinear part.')
    parser.add_argument('--lr', type=float, default=0.0001,
                        help='Learning rate.')
    parser.add_argument('--per_test', type=int, default=20,
                        help='Learning rate.')   
    parser.add_argument('--topN', type=int, default=50,
                        help='Learning rate.')  
    return parser.parse_args(args=[])

In [3]:
import random
import numpy as np
import torch

def demonstrate_randomness(seed=2024):
    # 设置 Python 内置的随机种子
    random.seed(seed)    
    # 设置 Numpy 的随机种子
    np.random.seed(seed)
    
    # 设置 PyTorch 的随机种子
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    # 确保 PyTorch 的一致性行为
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 调用函数
demonstrate_randomness()


In [4]:
args = parse_args(name)
data = Dataset(args)
    

loading data: meta_data

loading data: interaction_data

split data



In [5]:
from utils.log import LOG, load_model, save_model
log = LOG(args)



In [6]:
train_data =  get_DataLoader(data.train,args.batch_size, seq_len=10)
valid_data =  get_DataLoader(data.valid,args.batch_size, seq_len=10,train_flag=0)
test_data =  get_DataLoader(data.test,args.batch_size, seq_len=10,train_flag=0)


Using time span 128
total session: 12054
Using time span 128
total session: 4018
Using time span 128
total session: 4019


In [7]:
from AttMix import AttMix
model = AttMix(data.n_item,args.hidden_factor,args.batch_size,args)
model = model.cuda()

In [8]:
from utils.utils import calculate_session_embs
BASE = AttMix
best_model_path_student = './best_model/'+'%s+AttMix_S'%args.dataset
#load Student model
model = BASE(data.n_item,args.hidden_factor,args.batch_size,args)    
load_model(model, best_model_path_student)
model = model.cuda()



model loaded from ./best_model/Games+AttMix_S


In [9]:
class Database_LLM(object):
    def __init__(self,path):
        #|session_id|rec_result|hint_position|
        self.dir = path
        self.exist_sessions()
    def exist_sessions(self):
        # 计算文件的行数
        self.exist_session = list()
        with open(self.dir, 'r', encoding='utf-8') as file:
            for i, line in enumerate(file):
                session_id, rec_result, hint_position = line.split('\t')
                self.exist_session.append(int(session_id))
    def write(self,session_id,rec_result,hint_position):
        with open(self.dir,'a') as f:
            f.write("%d\t%s\t%d\n"%(session_id,str(rec_result),hint_position))
        self.exist_sessions()
    def get_all_dict(self):
        table_rec = dict()
        table_hit = dict()
        with open(self.dir, 'r', encoding='utf-8') as file:
            for line in file:
                session_id,rec_result,hint_position =line.split('\t')
                table_rec[int(session_id)] = eval(rec_result)
                table_hit[int(session_id)] = eval(hint_position[:-1])
        return table_rec,table_hit
    def write_sub_database(self,sub_dir,session_id):
        with open(sub_dir,'a') as f:
            f.write("%d\n"%session_id)
    def read_sub_database(self,sub_dir,st=0,end=50000):
        r = set()
        with open(sub_dir) as f:
            for i,line in enumerate(f):
                if i >= st and i <= end:
                    r.add(int(line[:-1]))
        return r

In [10]:
path = './datasets/%s/%s_LLM_Database.data'%(args.dataset,args.dataset)
database = Database_LLM(path)

In [11]:
import pandas as pd
df = pd.read_pickle('./datasets/%s/%s_prediction.pkl'%(args.dataset,args.dataset))

In [12]:
sub_df = df[df['session_id'].isin(database.exist_session)]
table_rec,table_hit = database.get_all_dict()
sub_df['LLM_pred'] = [table_rec[session_id] for session_id in sub_df.session_id]
sub_df['LLM_hit'] = [table_hit[session_id] for session_id in sub_df.session_id]
print(len(sub_df))

5359


/tmp/ipykernel_4058025/4249407072.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['LLM_pred'] = [table_rec[session_id] for session_id in sub_df.session_id]
/tmp/ipykernel_4058025/4249407072.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['LLM_hit'] = [table_hit[session_id] for session_id in sub_df.session_id]


In [13]:
if ablation == 'TR':
    select_df = df
else:
    ablation_sessions = database.read_sub_database('./datasets/%s/%s_%s.base'%(args.dataset,args.dataset,ablation),0,499)
    select_df =  sub_df[sub_df['session_id'].isin(ablation_sessions)]
    select_df = select_df
    other_df = df[~df['session_id'].isin(ablation_sessions)]

In [14]:
len(select_df)

500

In [15]:

if ablation != 'TR':
    #k1 + k2
    print(len(select_df[select_df.LLM_hit<25])/len(select_df))
    #k2
    print(len(select_df[(select_df.difference <= np.median(select_df.difference.values)) & (select_df.LLM_hit<25)])/len(select_df))

0.506
0.506


In [16]:
train_aug = []
negative_sampler = {key:[] for key in set(select_df.session_id.values)}
for i in range(len(select_df)):
    line = select_df.iloc[i]
    session = line.feature
    session_id = line.session_id
    if ablation == 'TR':
        LLM_pred = line.teacher_top_item
        for j,item in enumerate(LLM_pred):
            train_aug.append([session,item,session_id])

    else:
        LLM_pred = line.LLM_pred
        for j,item in enumerate(LLM_pred):
            if j<= 5:
                train_aug.append([session,item,session_id])
            if j<= 15:
                train_aug.append([session,item,session_id])
            if j<= 25:
                train_aug.append([session,item,session_id])


In [17]:
train_aug = pd.DataFrame(train_aug,columns=data.train.columns)
print(len(train_aug))

22748


In [18]:
train_aug_data =  get_DataLoader(train_aug, 1024, seq_len=10)


Using time span 128
total session: 22748


In [ ]:
from tqdm import tqdm
from utils.evaluation import evaluate
from utils.log import load_model, save_model
import time
import sys
import torch

def to_tensor(var, device):
    var = torch.Tensor(var)
    var = var.to(device)
    return var.long()

optimizer = torch.optim.Adam(model.parameters(), lr = args.lr)#, weight_decay=args.weight_decay)
best_metric = 0
for iter, (targets, items, mask,session_id) in enumerate(train_aug_data):
    #训练
    model.train()
    optimizer.zero_grad()
    targets_cuda = to_tensor(targets,'cuda')
    items_cuda = to_tensor(items,'cuda')
    mask_cuda = to_tensor(mask,'cuda')
    # negative_cuda = to_tensor(data.hard_negative_sample(session_id,negative_sampler,10),'cuda')
    negative_cuda = to_tensor(data.uniform_negative_sample(targets_cuda,1),'cuda')
    user_eb, scores = model(items_cuda,mask_cuda)
    loss = model.loss(user_eb,targets_cuda,negative_cuda)
    loss.backward()
    optimizer.step()
    if iter % args.per_test  == 0:#
        start_time = time.time()
        print(iter)
        model.eval()
        metrics = evaluate(model, valid_data,15,args=args)
        log_str = 'iter: %d, train loss: %.4f' % (iter, loss) # 打印loss
        if metrics != {}:
            log_str += ', ' + ', '.join(['valid ' + key + ': %.6f' % value for key, value in metrics.items()])
        print(log_str)
        log.write_str(log_str)
        # 保存recall最佳的模型
        if 'recall' in metrics:
            recall = metrics['recall']
            if recall > best_metric:
                best_metric = recall
                save_model(model, log.best_model_path)
                trials = 0
            else:
                trials += 1
                args.patience = 20 #if args.dataset =='rocket' else 3 
                if trials > args.patience: # early stopping
                    print("early stopping!")
                    break
        # 每次test之后loss_sum置零
        total_loss = 0.0
        test_time = time.time()
        print("time interval: %.4f min" % ((test_time-start_time)/60.0))
        sys.stdout.flush()
    if iter >=  10000: # 超过最大迭代次数，退出训练
        break

load_model(model, log.best_model_path)
model.eval()

# 训练结束后用valid_data测试一次
metrics = evaluate(model, valid_data,50,args=args)
print(', '.join(['Valid ' + key + ': %.6f' % value for key, value in metrics.items()]))
# 训练结束后用test_data测试一次
print("Test result:")
metrics = evaluate(model, test_data,5,args=args)
for key, value in metrics.items():
    output = 'test ' + key + '@5' + '=%.6f' % value
    print(output)
    log.write_str(output)
metrics = evaluate(model, test_data,10,args=args)
for key, value in metrics.items():
    output = 'test ' + key + '@10' + '=%.6f' % value
    print(output)
    log.write_str(output)    

0
iter: 0, train loss: 0.7509, valid recall: 0.017422, valid ndcg: 0.006384
time interval: 0.0098 min
20
iter: 20, train loss: 0.7467, valid recall: 0.017422, valid ndcg: 0.006372
time interval: 0.0059 min
40
iter: 40, train loss: 0.7326, valid recall: 0.016924, valid ndcg: 0.006222
time interval: 0.0060 min


In [ ]:
model